In [2]:
!pip install beautifulsoup4
!pip install requests
!pip install pickle5  # In case pickle is not pre-installed
!pip install langchain
!pip install langchain-groq
!pip install faiss-cpu
!pip install huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255314 sha256=686a7b2c070c1805c05e18b44c73a1734932118d6281cab8f7db784870232936
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.6 MB/s eta 0:00:00


In [4]:
!pip install --upgrade langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.24
    Uninstalling langchain-core-0.3.24:
      Successfully uninstalled langchain-core-0.3.24
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling langchain-0.3.11:
      Successfully uninstalled langchain-0.3.11


In [6]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [9]:
from bs4 import BeautifulSoup
import requests
import os
import pickle
import time
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Initialize LLM
llm = ChatGroq(temperature=0, groq_api_key="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", model_name="llama-3.1-70b-versatile")

file_path = "faiss_store_openai.pkl"

# Function to scrape content from a website
def scrape_website(url):
    # Send a request to the website
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract text from all paragraph tags (you can adjust this as needed)
        paragraphs = soup.find_all('p')
        text = ' '.join([para.get_text() for para in paragraphs])
        return text
    else:
        print(f"Failed to retrieve the webpage: {url}")
        return ""

# Process websites after input
def process_websites(urls):
    all_text = ""

    # Scrape text from all websites
    for url in urls:
        print(f"Processing website: {url}")
        extracted_text = scrape_website(url)
        all_text += extracted_text + "\n"

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_text(all_text)

    # Create embeddings and vector store
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore_openai = FAISS.from_texts(text_chunks, embeddings)

    # Save FAISS index
    print("Embedding Vector Started Building...✅✅✅")
    time.sleep(2)

    # Save the FAISS index to a pickle file
    with open(file_path, "wb") as f:
        pickle.dump(vectorstore_openai, f)

    print("Text extracted and FAISS index saved.")

# User input for URLs
urls = input("Enter the website URLs (comma-separated): ").split(',')

# Run processing after URL input
process_websites(urls)

# Query input
query = input("Ask a Question: ")
if query:
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            vectorstore = pickle.load(f)
            chain = RetrievalQA.from_llm(llm=llm, retriever=vectorstore.as_retriever())

        # Get response
        result = chain.run(query)

        # Display answer
        print("Answer:")
        print(result)


Enter the website URLs (comma-separated): https://www.google.com/
Processing website: https://www.google.com/
Embedding Vector Started Building...✅✅✅
Text extracted and FAISS index saved.
Ask a Question: how many launguages does google support
Answer:
I don't know the exact number of languages Google supports.
